# 02. Learning LangGraph - Chat Executor

In [18]:
!pip install --quiet -U langchain langchain_openai langgraph langchainhub langchain_experimental

You should consider upgrading via the 'c:\users\hrint\documents\ai-poc's\ai\scripts\python.exe -m pip install --upgrade pip' command.


modified from https://github.com/langchain-ai/langgraph/blob/main/examples/chat_agent_executor_with_function_calling/base.ipynb

## The model

In [20]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

## Tools

In [21]:
import random
from langgraph.prebuilt import ToolExecutor
from langchain_core.tools import tool


# dummy output
spglobal_desc = """The annual return of the S&P 500 index can vary significantly from year to year, reflecting the performance of the 500 largest companies listed on stock exchanges in the United States. Historically, the average annual return of the S&P 500, including dividends, has been around 7-10% after adjusting for inflation. However, this average includes periods of significant growth as well as periods of decline.\n\nTo give you a specific figure for a particular year, I would need the year in question. Since my knowledge is up to date only until early 2023, I can provide information up until the end of 2022. For example, the annual return for the S&P 500 in 2021 was approximately 26.89%, which was an exceptional year for the index.\n\nFor the most current annual return figures, you would need to look up the latest financial data or consult financial news sources that provide updates on the S&P 500's performance. Keep in mind that past performance is not indicative of future results, and investing in the stock market always carries risk."""

@tool("simple_retriever", return_direct=True)
def simple_retrieval_tool(input: str) -> str:
    """Returns the answer based on a single document"""
    return spglobal_desc


@tool("multi_doc_retriever", return_direct=True)
def multi_doc_retrieval_tool(input: str) -> str:
    """Returns the answer based on information from multiple documents"""
    return spglobal_desc


@tool("lower_case", return_direct=True)
def to_lower_case(input: str) -> str:
  """Returns the input as all lower case."""
  return input.lower()


@tool("random_number", return_direct=True)
def random_number_maker(input: str) -> str:
    """Returns a random number between 0-100."""
    return random.randint(0, 100)


tools = [simple_retrieval_tool, multi_doc_retrieval_tool,
         random_number_maker, to_lower_case]


tool_executor = ToolExecutor(tools)

In [22]:
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

In [23]:
from langchain.tools.render import format_tool_to_openai_function

functions = [format_tool_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

## AgentState

In [24]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Nodes

In [25]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    print(f"The agent action is {action}")
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    print(f"The tool result is: {response}")
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

## Graph

In [26]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent` where we start
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

## Run it

In [27]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
# user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
# user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

The agent action is tool='random_number' tool_input={'input': ''}
The tool result is: 29
The agent action is tool='lower_case' tool_input={'input': 'twenty nine'}
The tool result is: twenty nine


{'messages': [SystemMessage(content='you are a helpful assistant'),
  HumanMessage(content='give me a random number and then write in words and make it lower case'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":""}', 'name': 'random_number'}}),
  FunctionMessage(content='29', name='random_number'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"twenty nine"}', 'name': 'lower_case'}}),
  FunctionMessage(content='twenty nine', name='lower_case'),
  AIMessage(content='The random number is 29, and in words, it is "twenty nine" in lower case.')]}

In [28]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
# user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
# user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

The agent action is tool='lower_case' tool_input={'input': 'Merlion'}
The tool result is: merlion


{'messages': [SystemMessage(content='you are a helpful assistant'),
  HumanMessage(content="plear write 'Merlion' in lower case"),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"Merlion"}', 'name': 'lower_case'}}),
  FunctionMessage(content='merlion', name='lower_case'),
  AIMessage(content='The word "Merlion" in lower case is "merlion".')]}

In [30]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
# user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
# user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
user_01 = HumanMessage(content="what is a S&P 500 index retruns?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

The agent action is tool='simple_retriever' tool_input={'input': 'S&P 500 index returns'}
The tool result is: The annual return of the S&P 500 index can vary significantly from year to year, reflecting the performance of the 500 largest companies listed on stock exchanges in the United States. Historically, the average annual return of the S&P 500, including dividends, has been around 7-10% after adjusting for inflation. However, this average includes periods of significant growth as well as periods of decline.

To give you a specific figure for a particular year, I would need the year in question. Since my knowledge is up to date only until early 2023, I can provide information up until the end of 2022. For example, the annual return for the S&P 500 in 2021 was approximately 26.89%, which was an exceptional year for the index.

For the most current annual return figures, you would need to look up the latest financial data or consult financial news sources that provide updates on the S

{'messages': [SystemMessage(content='you are a helpful assistant'),
  HumanMessage(content='what is a S&P 500 index retruns?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"S&P 500 index returns"}', 'name': 'simple_retriever'}}),
  FunctionMessage(content="The annual return of the S&P 500 index can vary significantly from year to year, reflecting the performance of the 500 largest companies listed on stock exchanges in the United States. Historically, the average annual return of the S&P 500, including dividends, has been around 7-10% after adjusting for inflation. However, this average includes periods of significant growth as well as periods of decline.\n\nTo give you a specific figure for a particular year, I would need the year in question. Since my knowledge is up to date only until early 2023, I can provide information up until the end of 2022. For example, the annual return for the S&P 500 in 2021 was approximately 26.89%, which was an exc

In [31]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
# user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
# user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
user_01 = HumanMessage(content="what is a S&P 500 index retruns and compare it with factor roator index?")

inputs = {"messages": [system_message, user_01]}

app.invoke(inputs)

The agent action is tool='multi_doc_retriever' tool_input={'input': 'S&P 500 index returns and factor rotation index comparison'}
The tool result is: The annual return of the S&P 500 index can vary significantly from year to year, reflecting the performance of the 500 largest companies listed on stock exchanges in the United States. Historically, the average annual return of the S&P 500, including dividends, has been around 7-10% after adjusting for inflation. However, this average includes periods of significant growth as well as periods of decline.

To give you a specific figure for a particular year, I would need the year in question. Since my knowledge is up to date only until early 2023, I can provide information up until the end of 2022. For example, the annual return for the S&P 500 in 2021 was approximately 26.89%, which was an exceptional year for the index.

For the most current annual return figures, you would need to look up the latest financial data or consult financial ne

{'messages': [SystemMessage(content='you are a helpful assistant'),
  HumanMessage(content='what is a S&P 500 index retruns and compare it with factor roator index?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"S&P 500 index returns and factor rotation index comparison"}', 'name': 'multi_doc_retriever'}}),
  FunctionMessage(content="The annual return of the S&P 500 index can vary significantly from year to year, reflecting the performance of the 500 largest companies listed on stock exchanges in the United States. Historically, the average annual return of the S&P 500, including dividends, has been around 7-10% after adjusting for inflation. However, this average includes periods of significant growth as well as periods of decline.\n\nTo give you a specific figure for a particular year, I would need the year in question. Since my knowledge is up to date only until early 2023, I can provide information up until the end of 2022. For example, the 